In [ ]:
# System
import gc
import os
import sys
from pathlib import Path

# Cpu
from collections import Counter
import numpy as np
import pandas as pd

# Data viz
from IPython.display import Image, display, HTML
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pprint
import seaborn as sns

plt.rcParams['legend.framealpha'] = 1
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'k'
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.facecolor'] = 'w'
plt.rcParams['legend.handlelength'] = 1
plt.rcParams['legend.handleheight'] = 1.125
plt.rcParams['figure.figsize'] = (16, 9)


display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
from contextlib import contextmanager
import time
@contextmanager
def timer(name, unit='s'):
    start = time.time()
    yield
    delta = time.time() - start
    if unit == 's':
        pass
    elif unit == 'min':
        delta /= 60
    else:
        raise NotImplementedError
    print('{}: {:.2f}{}'.format(name, delta, unit))

# Mean shift pipeline + save visualisations

In [ ]:
PROJECT_DIR = 

In [ ]:
root = Path(PROJECT_DIR)
raw = root / 'raw_data'
p = root / 'toto'
p.mkdir(exist_ok=True)
manuscripts = sorted(list(filter(lambda x: x.is_dir(), raw.glob('*'))))

In [ ]:
N_SAMPLE = 1000
MEANSHIFT_BW = 10
L_3D = 100
L_2D = 200

In [ ]:
from cv2 import cvtColor, COLOR_RGB2Lab, COLOR_Lab2RGB
import cv2

from collections import Counter
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
from sklearn.cluster import MeanShift
from matplotlib.colors import ListedColormap

for manuscript in manuscripts:
    print(manuscript.name)
    output_path = (p / manuscript.name)
    output_path.mkdir(exist_ok=True)
    illuminations = sorted(manuscript.glob('**/*.jpg'))
    with open(output_path / 'cluster_info.tsv', mode="w") as f:
        f.write("image_name\tbandwidth\tnb_cluster\tcluster_centers\n")

    for illu in illuminations:
        print('\t{}'.format(illu.name))
        i = Image.open(illu)
        arr = np.array(i)
        ab = np.reshape(cvtColor(arr, COLOR_RGB2Lab)[:,:,1:], (-1, 2)).tolist()
        df = pd.DataFrame(ab, columns=['a', 'b'])
        df['ab'] = list(map(tuple, df.values.tolist()))
        counts = df['ab'].value_counts()
        keys, values = counts.index.tolist(), counts.values
        colors = cvtColor(np.reshape(np.hstack([[[L_3D]]*len(keys), np.array(keys)]), (1, -1, 3)).astype('uint8'), COLOR_Lab2RGB)[0] / 255.


        # Triangulation surface from pixel_count
        ## Linear scale
        abc = np.hstack([np.array(keys), np.reshape(np.array(counts.values), (-1, 1))])
        x, y, z = abc[:, 0], abc[:, 1], abc[:, 2]

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(x, y, z, color=(0, 0, 0, 0.2), lw=1)
        ax.scatter(xs=x, ys=y, zs=z, s=20,  c=colors, lw=0)
        ax.set_xlabel('a')
        ax.set_ylabel('b')
        ax.set_zlabel('pixel_count')
        fig.tight_layout()
        (output_path / 'trisurf30').mkdir(exist_ok=True)
        ax.view_init(30, 30)
        fig.savefig(output_path / 'trisurf30' / illu.name)
        (output_path / 'trisurf60').mkdir(exist_ok=True)
        ax.view_init(30, 60)
        fig.savefig(output_path /  'trisurf60' / illu.name)
        plt.close()

        ## Logscale
        abc = np.hstack([np.array(keys), np.reshape(np.log10(counts.values), (-1, 1))])
        x, y, z = abc[:, 0], abc[:, 1], abc[:, 2]

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(x, y, z, color=(0, 0, 0, 0.2), lw=1)
        ax.scatter(xs=x, ys=y, zs=z, s=20,  c=colors, lw=0)
        ax.set_xlabel('a')
        ax.set_ylabel('b')
        ax.set_zlabel('log10(pixel_count)')
        fig.tight_layout()
        (output_path / 'trisurf30_log').mkdir(exist_ok=True)
        ax.view_init(30, 30)
        fig.savefig(output_path / 'trisurf30_log' / illu.name)
        (output_path / 'trisurf60_log').mkdir(exist_ok=True)
        ax.view_init(30, 60)
        fig.savefig(output_path /  'trisurf60_log' / illu.name)
        plt.close()

        # Gaussian KDE
        ## Linearscale
        kernel = gaussian_kde(dataset=np.swapaxes(np.asarray(keys), 0, 1), bw_method=0.1, weights=values)
        xmin, xmax = df['a'].min(), df['a'].max()
        ymin, ymax = df['b'].min(), df['b'].max()
        X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
        positions = np.vstack([X.ravel(), Y.ravel()])
        Z = np.reshape(kernel(positions).T, X.shape)
        colors = cvtColor(np.dstack([np.ones(X.shape, dtype=np.uint8)*L_3D, X, Y]).astype(np.uint8), COLOR_Lab2RGB) / 255.

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(X, Y, Z, rcount=50, ccount=50, facecolors=colors, shade=False, alpha=1, edgecolors='none', lw=0)
        ax.set_xlabel('a')
        ax.set_ylabel('b')
        ax.set_zlabel('z')
        fig.tight_layout()
        (output_path / 'kde30').mkdir(exist_ok=True)
        ax.view_init(30, 30)
        fig.savefig(output_path / 'kde30' / illu.name)
        (output_path / 'kde60').mkdir(exist_ok=True)
        ax.view_init(30, 60)
        fig.savefig(output_path /  'kde60' / illu.name)
        plt.close()

        ## Logscale
        kernel = gaussian_kde(dataset=np.swapaxes(np.asarray(keys), 0, 1), bw_method=0.1, weights=np.log10(values))
        xmin, xmax = df['a'].min(), df['a'].max()
        ymin, ymax = df['b'].min(), df['b'].max()
        X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
        positions = np.vstack([X.ravel(), Y.ravel()])
        Z = np.reshape(kernel(positions).T, X.shape)
        colors = cvtColor(np.dstack([np.ones(X.shape, dtype=np.uint8)*L_3D, X, Y]).astype(np.uint8), COLOR_Lab2RGB) / 255.

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(X, Y, Z, rcount=50, ccount=50, facecolors=colors, shade=False, alpha=1, edgecolors='none', lw=0)
        ax.set_xlabel('a')
        ax.set_ylabel('b')
        ax.set_zlabel('z')
        fig.tight_layout()
        (output_path / 'kde30_log').mkdir(exist_ok=True)
        ax.view_init(30, 30)
        fig.savefig(output_path / 'kde30_log' / illu.name)
        (output_path / 'kde60_log').mkdir(exist_ok=True)
        ax.view_init(30, 60)
        fig.savefig(output_path /  'kde60_log' / illu.name)
        plt.close()

        # Clustering with mean shift
        ## Adaptive bw
        bw = MEANSHIFT_BW
        sample = df.sample(N_SAMPLE) if (len(df) > N_SAMPLE) else df
        with timer('\t\tmean_shift fitting'):
            cluster = MeanShift(bandwidth=bw, n_jobs=-1).fit(sample[['a', 'b']])
        unique_labels = np.unique(cluster.labels_)
        if len(unique_labels) < 4:
            bw /= 2
            with timer('\t\tmean_shift fitting'):
                cluster = MeanShift(bandwidth=bw, n_jobs=-1).fit(sample[['a', 'b']])
            unique_labels = np.unique(cluster.labels_)

        with open(output_path / 'cluster_info.tsv', mode="a") as f:
            f.write("{}\t{}\t{}\t{}\n".format(illu.name, bw, len(unique_labels), cluster.cluster_centers_.tolist()))

        colors_list = cvtColor(np.reshape(np.hstack([[[L_3D]]*len(unique_labels), cluster.cluster_centers_]), (1, -1, 3)).astype('uint8'), COLOR_Lab2RGB)[0] / 255.
        cmap = ListedColormap(colors_list)
        colors = cmap(cluster.predict(keys))

        ## Pixel count visualisation - linear scale
        abc = np.hstack([np.array(keys), np.reshape(np.array(values), (-1, 1))])
        x, y, z = abc[:, 0], abc[:, 1], abc[:, 2]

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(x, y, z, color=(0, 0, 0, 0.2), lw=1)
        ax.scatter(xs=x, ys=y, zs=z, s=20,  c=colors, lw=0, depthshade=False)
        ax.set_xlabel('a')
        ax.set_ylabel('b')
        ax.set_zlabel('pixel_count')
        fig.tight_layout()
        (output_path / 'cluster30').mkdir(exist_ok=True)
        ax.view_init(30, 30)
        fig.savefig(output_path / 'cluster30' / illu.name)
        (output_path / 'cluster60').mkdir(exist_ok=True)
        ax.view_init(30, 60)
        fig.savefig(output_path /  'cluster60' / illu.name)
        plt.close()

        ## Pixel count visualisation - log scale
        abc = np.hstack([np.array(keys), np.reshape(np.log10(values), (-1, 1))])
        x, y, z = abc[:, 0], abc[:, 1], abc[:, 2]

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(x, y, z, color=(0, 0, 0, 0.2), lw=1)
        ax.scatter(xs=x, ys=y, zs=z, s=20,  c=colors, lw=0, depthshade=False)
        ax.set_xlabel('a')
        ax.set_ylabel('b')
        ax.set_zlabel('log10(pixel_count)')
        fig.tight_layout()
        (output_path / 'cluster30_log').mkdir(exist_ok=True)
        ax.view_init(30, 30)
        fig.savefig(output_path / 'cluster30_log' / illu.name)
        (output_path / 'cluster60_log').mkdir(exist_ok=True)
        ax.view_init(30, 60)
        fig.savefig(output_path /  'cluster60_log' / illu.name)
        plt.close()

        ## Original image visu
        colors_list = cvtColor(np.reshape(np.hstack([[[L_2D]]*len(unique_labels), cluster.cluster_centers_]), (1, -1, 3)).astype('uint8'), COLOR_Lab2RGB)[0] / 255.
        cmap = ListedColormap(colors_list)
        labels = cluster.predict(df[['a', 'b']]).reshape(arr.shape[:2])
        res_img = (cmap(labels)[:, :, :3] * 255).astype(np.uint8)
        contours = []
        for k in unique_labels:
            mask = (labels == k).astype(np.uint8)
            _, contour, _ = cv2.findContours(mask,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours += contour
        cv2.drawContours(res_img, contours, -1, (25, 25, 25), thickness=1)
        (output_path / 'cluster2d').mkdir(exist_ok=True)
        Image.fromarray(res_img).save(str(output_path / 'cluster2d' / illu.name))